# 1

贪心

同53题，\
最简单的是重复n遍，以不同的位置作为起始。

In [9]:
from typing import List


class Solution:
    def maxSubarraySumCircular(self, nums: List[int]) -> int:
        ans = nums[0]
        for i in range(len(nums)):
            tmp = nums[i]
            for range_x in [range(i+1, len(nums)), range(i)]:
                for j in range_x:
                    # print(j, end=' ')
                    tmp = max(tmp, 0)
                    tmp += nums[j]
                    ans = max(tmp, ans)
        return ans


if __name__ == '__main__':
    nums_list = [
        [1,-2,3,-2],
        [5,-3,5],
        [3,-1,2,-1],
        [3,-2,2,-3],
        [-2,-3,-1],
        [3, -3, 1, -3, 4, -1, 2, 1, -3, 4, -1],
    ]

    for nums in nums_list:
        print(Solution().maxSubarraySumCircular(nums))

3
10
4
3
-1
9


超时

# 2 ✘

为了联系首尾，在第一次遍历完，继续重头再遍历到重置点。

分情况讨论，特殊处理。

In [23]:
from typing import List


class Solution:
    def maxSubarraySumCircular(self, nums: List[int]) -> int:
        def max_or_min_subarray(nums, start, end, init_val, func=max):
            ans = init_val
            tmp = ans
            idx_start, idx_end = 0, 0
            for i in range(start, end):
                tmp = func(tmp, 0)
                idx_start = i if tmp == 0 else idx_start
                tmp += nums[i]
                ans = func(tmp, ans)
                idx_end = i if tmp == ans else idx_end
            return ans, (idx_start, idx_end), tmp
        
        ans, (idx_start, idx_end), ans_end = max_or_min_subarray(nums, 1, len(nums), nums[0], max)
        print('max:', ans, f'idx:{idx_start}-{idx_end}', 'tmp:', ans_end, end='\t')
        if idx_start == 0:
            min_ans, (idx_start2, idx_end2), _ = max_or_min_subarray(nums, 1, len(nums), nums[0], min)
            if ans > 0 and min_ans < 0: # ans>0除去最大和为负数的情况，min_ans<0除去数组全大于0的情况
                if idx_start <= idx_start2 and idx_end >= idx_end2:
                    ans -= min_ans
                else:   # 不在idx_start-idx_end之间，ans和ans_end-min_ans比
                    ans = max(ans, ans_end-min_ans)
            print('min:', min_ans)
        else:
            ans_start, _, _ = max_or_min_subarray(nums, 0, idx_start, ans_end, max)
            ans = max(ans, ans_start)
            print('start_max:', ans_start)
        return ans


if __name__ == '__main__':
    nums_list = [
        [1,-2,3,-2],
        [5,-3,5],
        [3,-1,2,-1],
        [3,-2,2,-3],
        [-2,-3,-1],
        [3, -3, 1, -3, 4, -1, 2, 1, -3, 4, -1],
        [-2],
        [3,1,3,2,6],
        [5,5,0,-5,3,-3,2],
        [0,5,8,-9,9,-7,3,-2],
    ]

    for nums in nums_list:
        print(Solution().maxSubarraySumCircular(nums))

max: 3 idx:2-2 tmp: 1	start_max: 2
3
max: 7 idx:0-2 tmp: 7	min: -3
10
max: 4 idx:0-2 tmp: 3	min: -1
4
max: 3 idx:0-2 tmp: 0	min: -3
3
max: -1 idx:2-2 tmp: -1	start_max: -1
-1
max: 7 idx:4-9 tmp: 6	start_max: 9
9
max: -2 idx:0-0 tmp: -2	min: -2
-2
max: 15 idx:0-4 tmp: 15	min: 1
15
max: 10 idx:0-2 tmp: 7	min: -5
12
max: 13 idx:1-4 tmp: 7	start_max: 7
13


101/110 cases passed

未全部通过，特殊处理太多，不够精简。

# 3

更换思路，分成2部分。

第一部分是从头开始。\
第二部分是除去最小和之外的两端。

![](../imgs/918.svg)

In [53]:
from typing import List


class Solution:
    def maxSubarraySumCircular(self, nums: List[int]) -> int:
        def max_or_min_subarray(nums, range_x, init_val, func=max):
            ans = init_val
            tmp = ans
            idx_start, idx_end = [], 0  # max/min的起点不确定，用列表存起来
            for i in range_x:
                tmp = func(tmp, 0)
                if tmp == 0:
                    idx_start.append(i)
                tmp += nums[i]
                ans = func(tmp, ans)
                idx_end = i if tmp == ans else idx_end
            idx_start = [x for x in idx_start if x<=idx_end]    # 挑出起点
            idx_start = idx_start[-1] if idx_start else 0
            return ans, (idx_start, idx_end)
        
        ans, _ = max_or_min_subarray(nums, range(1, len(nums)), nums[0])
        # print('ans1:', ans, end='\t')
        _, (idx_start, idx_end) = max_or_min_subarray(nums, range(1, len(nums)), nums[0], func=min)
        # print(f'min_ans:{_} idx:{idx_start}-{idx_end}', end='\t')
        range_x = list(range(idx_end+1, len(nums))) + list(range(idx_start))  # 两端
        if range_x:
            ans2, _ = max_or_min_subarray(nums, range_x[1:], nums[range_x[0]])
            ans = max(ans, ans2)
        # print(range_x)
        return ans


if __name__ == '__main__':
    nums_list = [
        [1,-2,3,-2],
        [5,-3,5],
        [3,-1,2,-1],
        [3,-2,2,-3],
        [-2,-3,-1],
        [3, -3, 1, -3, 4, -1, 2, 1, -3, 4, -1],
        [-2],
        [3,1,3,2,6],
        [5,5,0,-5,3,-3,2],
        [0,5,8,-9,9,-7,3,-2],
    ]

    for nums in nums_list:
        print(Solution().maxSubarraySumCircular(nums))

ans1: 3	min_ans:-2 idx:3-3	[0, 1, 2]
3
ans1: 7	min_ans:-3 idx:1-1	[2, 0]
10
ans1: 4	min_ans:-1 idx:3-3	[0, 1, 2]
4
ans1: 3	min_ans:-3 idx:3-3	[0, 1, 2]
3
ans1: -1	min_ans:-6 idx:0-2	[]
-1
ans1: 7	min_ans:-5 idx:1-3	[4, 5, 6, 7, 8, 9, 10, 0]
9
ans1: -2	min_ans:-2 idx:0-0	[]
-2
ans1: 15	min_ans:1 idx:1-1	[2, 3, 4, 0]
15
ans1: 10	min_ans:-5 idx:3-5	[6, 0, 1, 2]
12
ans1: 13	min_ans:-9 idx:3-3	[4, 5, 6, 7, 0, 1, 2]
16


110/110 cases passed (236 ms)\
Your runtime beats 15.79 % of python3 submissions\
Your memory usage beats 26.47 % of python3 submissions (18.4 MB)

time: O(n)

space: O(1)

> 空间本应该是O(1)，但这里range_x为了方便直接list，实际为O(n)。

# 3+

看了题解后，突然想到其实徒劳多做了一次range_x的寻找最大子序列和。\
连位置也不用记录了，代码也精简了。

In [4]:
from typing import List


class Solution:
    def maxSubarraySumCircular(self, nums: List[int]) -> int:
        def max_or_min_subarray(nums, func=max):
            ans = nums[0]
            tmp = ans
            for i in range(1, len(nums)):
                tmp = func(tmp, 0)
                tmp += nums[i]
                ans = func(tmp, ans)
            return ans
        
        ans = max_or_min_subarray(nums)
        if ans < 0:
            return ans
        ans_min = max_or_min_subarray(nums, func=min)
        ans = max(ans, sum(nums)-ans_min)
        # print(ans, ans_min)
        return ans


if __name__ == '__main__':
    nums_list = [
        [1,-2,3,-2],
        [5,-3,5],
        [3,-1,2,-1],
        [3,-2,2,-3],
        [-2,-3,-1],
        [3, -3, 1, -3, 4, -1, 2, 1, -3, 4, -1],
        [-2],
        [3,1,3,2,6],
        [5,5,0,-5,3,-3,2],
        [0,5,8,-9,9,-7,3,-2],
    ]

    for nums in nums_list:
        print(Solution().maxSubarraySumCircular(nums))

3
10
4
3
-1
9
-2
15
12
16


110/110 cases passed (148 ms)\
Your runtime beats 86.85 % of python3 submissions\
Your memory usage beats 78.51 % of python3 submissions (18 MB)

time: O(n)

space: O(1)